## Composite Types

We’ll talk about four such types - arrays, slices, maps, and structs.

## Arrays
An array is a fixed-length sequence of zero or more elements of a particular type. Because of their fixed length, arrays are rarely used directly in Go. Slices, which can grow and shrink, are much more versatile.

In [1]:
import "fmt"

var a [3]int             // array of 3 integers
fmt.Println(a[0])        // print the first element
fmt.Println(a[len(a)-1]) // print the last element,

0
0


2 <nil>

In [2]:
import "reflect"

reflect.TypeOf(a).String()

[3]int

In [8]:
names := [3]string{"Simon", "James", "Luke"}

fmt.Println(names)

[Simon James Luke]


19 <nil>

In [10]:
len(names)

3

In [12]:
for i:=0; i<len(names); i++ {
    fmt.Println(names[i])
}

Simon
James
Luke


In [13]:
for _, name := range names{
    fmt.Println(name)
}

Simon
James
Luke


In [14]:
type Currency int

const (
    USD Currency = iota
    EUR
    GBP
    RMB
)

symbol := [...]string{USD: "$", EUR: "€", GBP: "£", RMB: "¥"}

fmt.Println(RMB, symbol[RMB]) // "3 ¥"

3 ¥


5 <nil>

When a function is called, a copy of each argument value is assigned to the corresponding parameter variable, so the function receives a copy, not the original. Passing large arrays in this way can be inefficient, and any changes that the function makes to array elements affect only the copy, not the original.

In [5]:
names := [3]string{"Simon", "James", "Luke"}

func chgNames(names [3]string){
    names[0] = "Bob"
}

chgNames(names)

In [6]:
names  // nothing changed here... was passed by value

[Simon James Luke]

In [7]:
func ptrChgNames(names *[3]string){
    names[0] = "Bob"
}

ptrChgNames(&names)

In [8]:
names

[Bob James Luke]

## Slices

Slices represent variable-length sequences whose elements all have the same type.

Arrays and slices are intimately connected. A slice is a lightweight data structure that gives access to a subsequence (or perhaps all) of the elements of an array, which is known as the slice’s underlying array.

A slice has three components: a pointer, a length, and a capacity.

In [14]:
var s []int

for i := 1; i <= 10; i++ {
    s = append(s, i)
}

s

[1 2 3 4 5 6 7 8 9 10]

In [15]:
// reverse reverses a slice of ints in place.
func reverse(s []int) {
    for i, j := 0, len(s)-1; i < j; i, j = i+1, j-1 {
        s[i], s[j] = s[j], s[i]
    }
}

reverse(s)
s

[10 9 8 7 6 5 4 3 2 1]

In [16]:
reverse(s)
s

[1 2 3 4 5 6 7 8 9 10]

Unlike arrays, slices are not comparable, so we cannot use == to test whether two slices contain the same elements. The standard library provides the highly optimized bytes.Equal function for comparing two slices of bytes ([]byte), but for other types of slice, we must do the comparison ourselves.

In [23]:
func equal(x, y []string) bool {
    if len(x) != len(y) {
        return false
    }
    for i := range x {
        if x[i] != y[i] {
            return false
        }
    }
    return true
}


In [24]:
var s, t []string

equal(s, t)

true

In [25]:
s := append(s, "astr")

In [26]:
equal(s, t)

false

In [27]:
t := append(t, "astr")
equal(s, t)

true

In [31]:
var s []int

s

[]

In [32]:
len(s)

0

In [30]:
s == nil

true

The built-in function make creates a slice of a specified element type, length, and capacity. The capacity argument may be omitted, in which case the capacity equals the length.

In [38]:
var s []string
s

[]

In [33]:
s := make([]string, 10)
s

[         ]

In [36]:
s[9] == ""

true

If there is insufficient space for growth, append must allocate a new array big enough to hold the result, copy the values from x into it, then append the new element y.

## In-Place Slice Techniques

In [53]:
func nonempty(strings []string) []string {
    i := 0
    for _, s := range strings {
        if s != "" {
            strings[i] = s
            i++
        }
    }
    return strings[:i]
}


In [54]:
s := []string{"these", "are", "some", "", "strings"}
s

[these are some  strings]

In [55]:
len(s)

5

In [56]:
x := nonempty(s)
x

[these are some strings]

In [57]:
len(x)

4

## Maps
The hash table is one of the most ingenious and versatile of all data structures.

In [58]:
ages := map[string]int{
    "alice":   31,
    "charlie": 34,
}

ages

map[alice:31 charlie:34]

In [59]:
ages := make(map[string]int)
ages["alice"] = 31
ages["charlie"] = 34
ages

map[alice:31 charlie:34]

In [60]:
ages["bob"]++

In [61]:
ages

map[charlie:34 bob:1 alice:31]

In [63]:
import "fmt"

for name, age := range ages {
    fmt.Println(name, age)
}

alice 31
charlie 34
bob 1


In [64]:
// sorting
import "sort"

var names []string
for name := range ages {
    names = append(names, name)
}

sort.Strings(names)
for _, name := range names {
    fmt.Printf("%s\t%d\n", name, ages[name])
}


alice	31
bob	1
charlie	34


In [65]:
age, ok := ages["mike"]
ok

false

As with slices, maps cannot be compared to each other; the only legal comparison is with nil. To test whether two maps contain the same keys and the same associated values, we must write a loop.

In [74]:
func equal(x, y map[string]int) bool {
    if len(x) != len(y) {
        return false
    }
    for k, xv := range x {
        if yv, ok := y[k]; !ok || yv != xv {
            return false
        }
    }
    return true
}

In [70]:
x := new(map [string]int)
y := new(map [string]int)

In [72]:
import "reflect"

reflect.TypeOf(x).String()  // pointer

*map[string]int

In [75]:
equal(*x, *y)

true

Go does not provide a set type, but since the keys of a map are distinct, a map can serve this purpose.

In [76]:
set := make(map[int]bool)
set

map[]

In [79]:
values := []int{1, 1, 2, 2, 3, 3, 3}
values

[1 1 2 2 3 3 3]

In [80]:
for _, value := range values {
    set[value] = true
}

In [81]:
set

map[1:true 2:true 3:true]

In [83]:
for index, _ := range set {
    fmt.Println(index)
}

1
2
3


## Structs

A struct is an aggregate data type that groups together zero or more named values of arbitrary types as a single entity. Each value is called a field.

In [84]:
type Employee struct {
    ID       int
    Name     string
    Position string
}

In [86]:
e := new(Employee)
e

&{0  }

In [87]:
e.ID

0

In [89]:
e.Name == ""

true

In [90]:
e.Position == ""

true

In [97]:
p := &e  // pointer to employee
p

0xc00000e530

In [103]:
(*p).ID

0

In [105]:
(*p).Name == ""

true

In [106]:
e := Employee{7, "Simon", "Janitor"}

In [109]:
fmt.Println(e.ID, e.Name, e.Position)

7 Simon Janitor


16 <nil>

In [110]:
p := &e

In [111]:
fmt.Println((*p).ID, (*p).Name, (*p).Position)

7 Simon Janitor


16 <nil>

The name of a struct field is exported if it begins with a capital letter; this is Go’s main access control mechanism. A struct type may contain a mixture of exported and unexported fields.

The individual fields of a struct are accessed using dot notation. This will also work with struct pointers.

If a field is omitted in this kind of literal, it is set to the zero value for its type.

In [1]:
type Point struct{ X, Y int }

In [2]:
p := Point{1, 2}
p

{1 2}

In [3]:
q := new(Point)
q

&{0 0}

In [4]:
pp := &p
pp

&{1 2}

Struct comparison...

In [5]:
p := Point{1, 2}
q := Point{1, 2}

In [6]:
p == q

true

In [7]:
p.X = 7
p == q

false

## Struct Embedding

In [9]:
type Point struct { X, Y int}

type Circle struct {
    Center Point
    Radius int
}

type Wheel struct {
    Circle Circle
    Spokes int
}


In [11]:
// does make accessing the fields of wheel more verbose
var w Wheel
w.Circle.Center.X = 8
w.Circle.Center.Y = 8
w.Circle.Radius = 5
w.Spokes = 20

Go lets us declare a field with a type but no name; such fields are called anonymous fields. The type of the field must be a named type or a pointer to a named type.

In [12]:
type Circle struct {
    Point
    Radius int
}

type Wheel struct {
    Circle
    Spokes int
}


In [13]:
var w Wheel
w.X = 8        // equivalent to w.Circle.Point.X = 8
w.Y = 8        // equivalent to w.Circle.Point.Y = 8
w.Radius = 5   // equivalent to w.Circle.Radius = 5
w.Spokes = 20

## JSON
JavaScript Object Notation (JSON) is a standard notation for sending and receiving structured information. JSON is not the only such notation. XML, ASN.1, and Google’s Protocol Buffers serve similar purposes and each has its niche, but because of its simplicity, readability, and universal support, JSON is the most widely used.

Go has excellent support for encoding and decoding these formats, provided by the standard library packages encoding/json, encoding/xml, encoding/asn1, and so on.

In [14]:
type Movie struct {
    Title  string
    Year   int  `json:"released"`
    Color  bool `json:"color,omitempty"`
    Actors []string
}


In [15]:
var movies = []Movie{
    {Title: "Casablanca", Year: 1942, Color: false,
        Actors: []string{"Humphrey Bogart", "Ingrid Bergman"}},
    {Title: "Cool Hand Luke", Year: 1967, Color: true,
        Actors: []string{"Paul Newman"}},
    {Title: "Bullitt", Year: 1968, Color: true,
        Actors: []string{"Steve McQueen", "Jacqueline Bisset"}},
}


In [16]:
movies

[{Casablanca 1942 false [Humphrey Bogart Ingrid Bergman]} {Cool Hand Luke 1967 true [Paul Newman]} {Bullitt 1968 true [Steve McQueen Jacqueline Bisset]}]

In [19]:
import (
    "fmt"
    "encoding/json"
    "log"
)

data, err := json.Marshal(movies)
if err != nil {
    log.Fatalf("JSON marshaling failed: %s", err)
}
fmt.Printf("%s\n", data)


[{"Title":"Casablanca","released":1942,"Actors":["Humphrey Bogart","Ingrid Bergman"]},{"Title":"Cool Hand Luke","released":1967,"color":true,"Actors":["Paul Newman"]},{"Title":"Bullitt","released":1968,"color":true,"Actors":["Steve McQueen","Jacqueline Bisset"]}]


264 <nil>

In [21]:
import "reflect"

reflect.TypeOf(data).String()

[]uint8

In [22]:
data, err := json.MarshalIndent(movies, "", "    ")
if err != nil {
    log.Fatalf("JSON marshaling failed: %s", err)
}
fmt.Printf("%s\n", data)


[
    {
        "Title": "Casablanca",
        "released": 1942,
        "Actors": [
            "Humphrey Bogart",
            "Ingrid Bergman"
        ]
    },
    {
        "Title": "Cool Hand Luke",
        "released": 1967,
        "color": true,
        "Actors": [
            "Paul Newman"
        ]
    },
    {
        "Title": "Bullitt",
        "released": 1968,
        "color": true,
        "Actors": [
            "Steve McQueen",
            "Jacqueline Bisset"
        ]
    }
]


497 <nil>

Only exported fields are marshaled, which is why we chose capitalized names for all the Go field names.

In [28]:
results := make([]Movie, 0)
err := json.Unmarshal(data, &results)
if err != nil {
    log.Fatalf("JSON unmarshaling failed: %s", err)
}

results

[{Casablanca 1942 false [Humphrey Bogart Ingrid Bergman]} {Cool Hand Luke 1967 true [Paul Newman]} {Bullitt 1968 true [Steve McQueen Jacqueline Bisset]}]

## Text and HTML templates
the text/template and html/template packages, which provide a mechanism for substituting the values of variables into a text or HTML template.

A template is a string or file containing one or more portions enclosed in double braces, {{...}}, called actions.

In [4]:
import (
    "os"
    "log"
    "text/template"
)

type Todo struct {
    Name        string
    Description string
}

func main() {
    td := Todo{"Test templates", "Let's test a template to see the magic."}
    
    t, err := template.New("todos").Parse("You have a task named \"{{ .Name}}\" with description: \"{{ .Description}}\"")
    if err != nil {
        panic(err)
    }
    err = t.Execute(os.Stdout, td)
    if err != nil {
        panic(err)
    }
}

main()

You have a task named "Test templates" with description: "Let's test a template to see the magic."

Testing this out with a number of templates...

In [5]:
todos := []Todo{
    Todo{"Test templates", "Let's test a template to see the magic."},
    Todo{"Munch", "Eat leftovers in the fridge."},
    Todo{"Read", "Go through your reading list."},
}

todos

[{Test templates Let's test a template to see the magic.} {Munch Eat leftovers in the fridge.} {Read Go through your reading list.}]

***